In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/code/data/train/subtask_1/en/')

Mounted at /content/drive


In [ ]:
%pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import random
import re
from sklearn.metrics import f1_score, accuracy_score
import transformers
import torch
import openai

In [ ]:
df_human = pd.read_csv("human.csv")
df_bloom1b7 = pd.read_csv("bloom1b7.csv")
df_bloom3b = pd.read_csv("bloom3b.csv")
df_bloom7b1 = pd.read_csv("bloom7b1.csv")
df_gpt = pd.read_csv("gpt.csv")

In [ ]:
from google.colab import userdata
api_key = userdata.get('api-key')

In [ ]:
def classify_texts(df, prompt_template, output_path):
    openai.api_key = api_key

    # Initialize a list to store the results
    classification_results = []

    # Loop through each text in the dataset
    for index, row in df.iterrows():
        text = row['text']
        label = row['label']
        domain = row['domain']
        text_id = row['id']
        text_to_classify = row['text']

        # Create prompt message based on template
        prompt = prompt_template.format(text_to_classify=text_to_classify)

        completion = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a classification assistant."},
                {"role": "user", "content": prompt},
            ]
        )

        # Extract the response
        generated_text = completion.choices[0].message.content

        classification_results.append({'id': text_id, 'text': text, 'generated_label': generated_text, 'label': label, 'domain': domain})

    # Convert the results into a DataFrame
    classification_df = pd.DataFrame(classification_results)
    classification_df.to_csv(output_path, index=False)

In [ ]:
prompt = """Rate the likelihood of this text being generated by a large language model on a scale of 1 (very unlikely) to 5 (very likely). Please provide your conclusion: was this text likely generated by a machine or a human?
Text: {text_to_classify}
Response format: Likelihood, Machine or Human
"""

In [ ]:
classify_texts(df_human, prompt, "classification_results_gpt_human.csv")

In [ ]:
classify_texts(df_bloom1b7, prompt, "classification_results_gpt_1b7.csv")

In [ ]:
classify_texts(df_bloom3b, prompt, "classification_results_gpt_3b.csv")

In [ ]:
classify_texts(df_gpt, prompt, "classification_results_gpt_gpt.csv")

In [ ]:
import pandas as pd
from sklearn.metrics import classification_report, f1_score

def clean_generated_text(generated_text):
    if 'human' in generated_text.lower():
        return 'human'
    elif 'machine' in generated_text.lower():
        return 'generated'
    #elif 'generated' in generated_text.lower():
    #    return 'machine'
    elif 'AI' in generated_text:
        return 'generated'
    elif 'large language model' in generated_text.lower():
        return 'generated'
    else:
        return None

def results_evaluation(file_path, correct_answer):
    df = pd.read_csv(file_path)

    # Extract labels from the text
    df['generated_label'] = df['generated_label'].apply(clean_generated_text)

    # Drop rows with missing or incorrect labels
    df.dropna(subset=['generated_label'], inplace=True)
    df = df[df['generated_label'].isin(['generated', 'human'])]

    df['correct_answer'] = correct_answer

    report = classification_report(df['correct_answer'], df['generated_label'], zero_division=1)

    micro_f1 = f1_score(df['correct_answer'], df['generated_label'], average='micro')

    print(df['generated_label'].value_counts())

    return micro_f1, report

#  Results

In [ ]:
report, f1 = results_evaluation("classification_results_gpt_human.csv", "human" )
print(report)

print(f1)

generated_label
human        551
generated    449
Name: count, dtype: int64
0.551
              precision    recall  f1-score   support

   generated       0.00      1.00      0.00         0
       human       1.00      0.55      0.71      1000

    accuracy                           0.55      1000
   macro avg       0.50      0.78      0.36      1000
weighted avg       1.00      0.55      0.71      1000



In [ ]:
report, f1 = results_evaluation("classification_results_gpt_1b7.csv", "generated" )
print(report)

print(f1)

generated_label
generated    663
human        337
Name: count, dtype: int64
0.663
              precision    recall  f1-score   support

   generated       1.00      0.66      0.80      1000
       human       0.00      1.00      0.00         0

    accuracy                           0.66      1000
   macro avg       0.50      0.83      0.40      1000
weighted avg       1.00      0.66      0.80      1000



In [ ]:
report, f1 = results_evaluation("classification_results_gpt_3b.csv", "generated" )
print(report)

print(f1)

generated_label
generated    568
human        432
Name: count, dtype: int64
0.568
              precision    recall  f1-score   support

   generated       1.00      0.57      0.72      1000
       human       0.00      1.00      0.00         0

    accuracy                           0.57      1000
   macro avg       0.50      0.78      0.36      1000
weighted avg       1.00      0.57      0.72      1000



In [ ]:
report, f1 = results_evaluation("classification_results_gpt_gpt.csv", "generated" )
print(report)

print(f1)

generated_label
human        663
generated    337
Name: count, dtype: int64
0.337
              precision    recall  f1-score   support

   generated       1.00      0.34      0.50      1000
       human       0.00      1.00      0.00         0

    accuracy                           0.34      1000
   macro avg       0.50      0.67      0.25      1000
weighted avg       1.00      0.34      0.50      1000

